In [1]:
import time
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.preprocessing import LabelEncoder
from itertools import product
import lightgbm as lgb
from sklearn.metrics import f1_score, roc_auc_score
import gc
import warnings
warnings.filterwarnings('ignore')

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', 200)

In [2]:
# count编码
def count_coding(df, fea_col):
    for f in fea_col:
        df[f + '_count'] = df[f].map(df[f].value_counts())
    return df

# 交叉特征统计
def cross_cat_num(df, num_col, cat_col):
    for f1 in tqdm(cat_col):
        g = df.groupby(f1, as_index=False)
        for f2 in tqdm(num_col):
            feat = g[f2].agg({
                '{}_{}_max'.format(f1, f2): 'max',
                '{}_{}_min'.format(f1, f2): 'min',
                '{}_{}_median'.format(f1, f2): 'median'
            })
            df = df.merge(feat, on=f1, how='left')
    return df

In [3]:
class MeanEncoder:
    def __init__(self, categorical_features, n_splits=10, target_type='classification', prior_weight_func=None):
        """
        :param categorical_features: list of str, the name of the categorical columns to encode
 
        :param n_splits: the number of splits used in mean encoding
 
        :param target_type: str, 'regression' or 'classification'
 
        :param prior_weight_func:
        a function that takes in the number of observations, and outputs prior weight
        when a dict is passed, the default exponential decay function will be used:
        k: the number of observations needed for the posterior to be weighted equally as the prior
        f: larger f --> smaller slope
        """
 
        self.categorical_features = categorical_features
        self.n_splits = n_splits
        self.learned_stats = {}
 
        if target_type == 'classification':
            self.target_type = target_type
            self.target_values = []
        else:
            self.target_type = 'regression'
            self.target_values = None
 
        if isinstance(prior_weight_func, dict):
            self.prior_weight_func = eval('lambda x: 1 / (1 + np.exp((x - k) / f))', dict(prior_weight_func, np=np))
        elif callable(prior_weight_func):
            self.prior_weight_func = prior_weight_func
        else:
            self.prior_weight_func = lambda x: 1 / (1 + np.exp((x - 2) / 1))
 
    @staticmethod
    def mean_encode_subroutine(X_train, y_train, X_test, variable, target, prior_weight_func):
        X_train = X_train[[variable]].copy()
        X_test = X_test[[variable]].copy()
 
        if target is not None:
            nf_name = '{}_pred_{}'.format(variable, target)
            X_train['pred_temp'] = (y_train == target).astype(int)  # classification
        else:
            nf_name = '{}_pred'.format(variable)
            X_train['pred_temp'] = y_train  # regression
        prior = X_train['pred_temp'].mean()
 
        col_avg_y = X_train.groupby(by=variable, axis=0)['pred_temp'].agg(['mean', 'size'])
        col_avg_y['size'] = prior_weight_func(col_avg_y['size'])
        col_avg_y[nf_name] = col_avg_y['size'] * prior + (1 - col_avg_y['size']) * col_avg_y['mean']
        col_avg_y.drop(['size', 'mean'], axis=1, inplace=True)
 
        nf_train = X_train.join(col_avg_y, on=variable)[nf_name].values
        nf_test = X_test.join(col_avg_y, on=variable).fillna(prior, inplace=False)[nf_name].values
 
        return nf_train, nf_test, prior, col_avg_y
 
    def fit_transform(self, X, y):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :param y: pandas Series or numpy array, n_samples
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
        if self.target_type == 'classification':
            skf = StratifiedKFold(self.n_splits)
        else:
            skf = KFold(self.n_splits)
 
        if self.target_type == 'classification':
            self.target_values = sorted(set(y))
            self.learned_stats = {'{}_pred_{}'.format(variable, target): [] for variable, target in
                                  product(self.categorical_features, self.target_values)}
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, target, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        else:
            self.learned_stats = {'{}_pred'.format(variable): [] for variable in self.categorical_features}
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new.loc[:, nf_name] = np.nan
                for large_ind, small_ind in skf.split(X, y):
                    nf_large, nf_small, prior, col_avg_y = MeanEncoder.mean_encode_subroutine(
                        X_new.iloc[large_ind], y.iloc[large_ind], X_new.iloc[small_ind], variable, None, self.prior_weight_func)
                    X_new.iloc[small_ind, -1] = nf_small
                    self.learned_stats[nf_name].append((prior, col_avg_y))
        return X_new
 
    def transform(self, X):
        """
        :param X: pandas DataFrame, n_samples * n_features
        :return X_new: the transformed pandas DataFrame containing mean-encoded categorical features
        """
        X_new = X.copy()
 
        if self.target_type == 'classification':
            for variable, target in product(self.categorical_features, self.target_values):
                nf_name = '{}_pred_{}'.format(variable, target)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
        else:
            for variable in self.categorical_features:
                nf_name = '{}_pred'.format(variable)
                X_new[nf_name] = 0
                for prior, col_avg_y in self.learned_stats[nf_name]:
                    X_new[nf_name] += X_new[[variable]].join(col_avg_y, on=variable).fillna(prior, inplace=False)[
                        nf_name]
                X_new[nf_name] /= self.n_splits
 
        return X_new

### 用户表

In [4]:
train_user = pd.read_hdf('../input/train_user.h5')
test_user = pd.read_hdf('../input/test_user.h5')

In [5]:
df_user = pd.concat([train_user, test_user])

In [6]:
# 号码量*消费值
df_user['idcard_cnt*arpu_202004'] = df_user['idcard_cnt'] * df_user['arpu_202004']

In [7]:
# count编码
count_list = ['city_name', 'county_name', 'idcard_cnt']

df_user = count_coding(df_user, count_list)

In [8]:
lbl = LabelEncoder()

for f in tqdm(['city_name', 'county_name']):
    df_user[f] = df_user[f].fillna('NA')
    df_user[f] = lbl.fit_transform(df_user[f].astype(str))

100%|███████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 222.39it/s]


In [9]:
# 交叉特征
cross_cat = ['city_name', 'county_name']
cross_num = ['idcard_cnt']
df_user = cross_cat_num(df_user, cross_num, cross_cat)

100%|████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 44.47it/s]


### 电话表

In [10]:
cols = ['phone_no_m', 'opposite_no_m', 'calltype_id', 'start_datetime', 'call_dur', 'imei_m']

train_voc = pd.read_hdf('../input/train_voc.h5', usecols=cols)
test_voc = pd.read_hdf('../input/test_voc.h5', usecols=cols)

In [11]:
# data['time']=(data['day'].values - data['day'].min()) * 24  + data['hour'].values

train_voc['voc_day'] = train_voc['start_datetime'].astype('datetime64').dt.day
train_voc['voc_hour'] = train_voc['start_datetime'].astype('datetime64').dt.hour
train_voc['voc_dayofweek'] = train_voc['start_datetime'].astype('datetime64').dt.dayofweek
train_voc['voc_time'] = (train_voc['voc_day'] - train_voc['voc_day'].min()) * 24 + train_voc['voc_hour']

test_voc['voc_day'] = test_voc['start_datetime'].astype('datetime64').dt.day
test_voc['voc_hour'] = test_voc['start_datetime'].astype('datetime64').dt.hour
test_voc['voc_dayofweek'] = test_voc['start_datetime'].astype('datetime64').dt.dayofweek
test_voc['voc_time'] = (test_voc['voc_day'] - test_voc['voc_day'].min()) * 24 + test_voc['voc_hour']

In [12]:
df_voc = pd.concat([train_voc, test_voc])

In [13]:
# 每个phone_no_m有几个独立的手机
df_voc['voc_phone_nunique'] = df_voc.groupby('phone_no_m')['imei_m'].transform('nunique')

# 每个phone_no_m在整个区间内与几个不同的人通话
df_voc['voc_phone_nunique'] = df_voc.groupby('phone_no_m')['opposite_no_m'].transform('nunique')

# 该月总的通话次数
df_voc['voc_count'] = df_voc.groupby('phone_no_m')['phone_no_m'].transform('count')

# 该月总的通话时长
df_voc['call_dur_sum'] = df_voc.groupby('phone_no_m')['call_dur'].transform('sum')

# 相互打电话次数
df_voc['voc_count_mutual'] = df_voc.groupby(['phone_no_m', 'opposite_no_m'])['phone_no_m'].transform('count')

# 相互打电话总时长
df_voc['call_dur_sum_mutual'] = df_voc.groupby(['phone_no_m', 'opposite_no_m'])['call_dur'].transform('sum')

#### 通话次数

##### 每天通话次数统计 day

In [14]:
## 用户每个小时,每天出现的次数

# for time in ['day','time']:
#     print('user_id_'+time)
#     data['user_id_'+time] = data['user_id']  + '_' + data[time].astype(str)
#     dic_ = data['user_id_'+time].value_counts().to_dict()
#     data['user_id_'+time +'_count'] = data['user_id_'+time].apply(lambda x: dic_[x])
#     data.drop('user_id_'+time, axis=1,inplace = True)

In [15]:
# 每天通话次数统计
df_voc['voc_day_count'] = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].transform('count')
df_voc['voc_day_count_max'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('max')
df_voc['voc_day_count_min'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('min')
df_voc['voc_day_count_mean'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('mean')
df_voc['voc_day_count_std'] = df_voc.groupby('phone_no_m')['voc_day_count'].transform('std')


# 每天与不同的人通话次数统计
df_voc['voc_day_diff_count'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_day'])['phone_no_m'].transform('count')
df_voc['voc_day_diff_count_max'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('max')
df_voc['voc_day_diff_count_min'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('min')
df_voc['voc_day_diff_count_mean'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('mean')
df_voc['voc_day_diff_count_std'] = df_voc.groupby('phone_no_m')['voc_day_diff_count'].transform('std')

del df_voc['voc_day_count'], df_voc['voc_day_diff_count']
gc.collect()

# 每个phone_no_m在几号的通话次数
voc_day_res = df_voc.groupby(['phone_no_m', 'voc_day'])['phone_no_m'].count().unstack()

for i in df_voc['voc_day'].unique():
    df_voc['voc_day{}_count'.format(i)] = df_voc['phone_no_m'].map(voc_day_res[i])

##### 小时通话次数统计 voc_time

In [16]:
# 小时通话次数统计
df_voc['voc_time_count'] = df_voc.groupby(['phone_no_m', 'voc_time'])['phone_no_m'].transform('count')
df_voc['voc_time_count_max'] = df_voc.groupby('phone_no_m')['voc_time_count'].transform('max')
df_voc['voc_time_count_min'] = df_voc.groupby('phone_no_m')['voc_time_count'].transform('min')
df_voc['voc_time_count_mean'] = df_voc.groupby('phone_no_m')['voc_time_count'].transform('mean')
df_voc['voc_time_count_std'] = df_voc.groupby('phone_no_m')['voc_time_count'].transform('std')


# 小时与不同的人通话次数统计
df_voc['voc_time_diff_count'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_time'])['phone_no_m'].transform('count')
df_voc['voc_time_diff_count_max'] = df_voc.groupby('phone_no_m')['voc_time_diff_count'].transform('max')
df_voc['voc_time_diff_count_min'] = df_voc.groupby('phone_no_m')['voc_time_diff_count'].transform('min')
df_voc['voc_time_diff_count_mean'] = df_voc.groupby('phone_no_m')['voc_time_diff_count'].transform('mean')
df_voc['voc_time_diff_count_std'] = df_voc.groupby('phone_no_m')['voc_time_diff_count'].transform('std')

del df_voc['voc_time_count'], df_voc['voc_time_diff_count']
gc.collect()

105

##### 每小时通话次数统计 hour

In [17]:
# 一小时内通话次数

df_voc['voc_hour_count'] = df_voc.groupby(['phone_no_m', 'voc_hour'])['phone_no_m'].transform('count')
df_voc['voc_hour_count_max'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('max')
df_voc['voc_hour_count_min'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('min')
df_voc['voc_hour_count_mean'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('mean')
df_voc['voc_hour_count_std'] = df_voc.groupby('phone_no_m')['voc_hour_count'].transform('std')


# 一小时内与不同的人通话次数

df_voc['voc_hour_diff_count'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_hour'])['phone_no_m'].transform('count')
df_voc['voc_hour_diff_count_max'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('max')
df_voc['voc_hour_diff_count_min'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('min')
df_voc['voc_hour_diff_count_mean'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('mean')
df_voc['voc_hour_diff_count_std'] = df_voc.groupby('phone_no_m')['voc_hour_diff_count'].transform('std')

del df_voc['voc_hour_count'], df_voc['voc_hour_diff_count']
gc.collect()

104

##### 按周几通话次数统计

In [18]:
# 周几通话次数

df_voc['voc_dayofweek_count'] = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['phone_no_m'].transform('count')
df_voc['voc_dayofweek_count_max'] = df_voc.groupby('phone_no_m')['voc_dayofweek_count'].transform('max')
df_voc['voc_dayofweek_count_min'] = df_voc.groupby('phone_no_m')['voc_dayofweek_count'].transform('min')
df_voc['voc_dayofweek_count_mean'] = df_voc.groupby('phone_no_m')['voc_dayofweek_count'].transform('mean')
df_voc['voc_dayofweek_count_std'] = df_voc.groupby('phone_no_m')['voc_dayofweek_count'].transform('std')


# 周几与不同的人通话次数

df_voc['voc_dayofweek_diff_count'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_dayofweek'])['phone_no_m'].transform('count')
df_voc['voc_dayofweek_diff_count_max'] = df_voc.groupby('phone_no_m')['voc_dayofweek_diff_count'].transform('max')
df_voc['voc_dayofweek_diff_count_min'] = df_voc.groupby('phone_no_m')['voc_dayofweek_diff_count'].transform('min')
df_voc['voc_dayofweek_diff_count_mean'] = df_voc.groupby('phone_no_m')['voc_dayofweek_diff_count'].transform('mean')
df_voc['voc_dayofweek_diff_count_std'] = df_voc.groupby('phone_no_m')['voc_dayofweek_diff_count'].transform('std')

del df_voc['voc_dayofweek_count'], df_voc['voc_dayofweek_diff_count']
gc.collect()

# 每个phone_no_m在周几的通话次数
voc_dayofweek_res = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['phone_no_m'].count().unstack()

for i in df_voc['voc_dayofweek'].unique():
    df_voc['voc_dayofweek{}_count'.format(i)] = df_voc['phone_no_m'].map(voc_dayofweek_res[i])

#### 通话时长

##### 每天通话时长统计 voc_day

In [19]:
# 每天通话时长统计
df_voc['call_dur_day_sum'] = df_voc.groupby(['phone_no_m', 'voc_day'])['call_dur'].transform('sum')
df_voc['call_dur_day_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('max')
df_voc['call_dur_day_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('min')
df_voc['call_dur_day_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('mean')
df_voc['call_dur_day_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_day_sum'].transform('std')


# 每天与不同的人通话时长统计
df_voc['call_dur_day_diff_sum'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_day'])['call_dur'].transform('sum')
df_voc['call_dur_day_diff_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_day_diff_sum'].transform('max')
df_voc['call_dur_day_diff_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_day_diff_sum'].transform('min')
df_voc['call_dur_day_diff_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_day_diff_sum'].transform('mean')
df_voc['call_dur_day_diff_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_day_diff_sum'].transform('std')

del df_voc['call_dur_day_sum'], df_voc['call_dur_day_diff_sum']
gc.collect()


# 每个phone_no_m在几号的通话时长
voc_day_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_day'])['call_dur'].sum().unstack()

for i in df_voc['voc_day'].unique():
    df_voc['voc_day{}_call_dur_sum'.format(i)] = df_voc['phone_no_m'].map(voc_day_call_dur_res[i])

##### 小时通话时长统计 voc_time

In [20]:
# 小时通话时长统计
df_voc['call_dur_time_sum'] = df_voc.groupby(['phone_no_m', 'voc_time'])['call_dur'].transform('sum')
df_voc['call_dur_time_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_time_sum'].transform('max')
df_voc['call_dur_time_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_time_sum'].transform('min')
df_voc['call_dur_time_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_time_sum'].transform('mean')
df_voc['call_dur_time_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_time_sum'].transform('std')


# 小时与不同的人通话时长统计
df_voc['call_dur_time_diff_sum'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_time'])['call_dur'].transform('sum')
df_voc['call_dur_time_diff_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_time_diff_sum'].transform('max')
df_voc['call_dur_time_diff_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_time_diff_sum'].transform('min')
df_voc['call_dur_time_diff_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_time_diff_sum'].transform('mean')
df_voc['call_dur_time_diff_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_time_diff_sum'].transform('std')

del df_voc['call_dur_time_sum'], df_voc['call_dur_time_diff_sum']
gc.collect()


# 每个phone_no_m在几号的通话时长
# voc_day_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_day'])['call_dur'].sum().unstack()

# for i in df_voc['voc_day'].unique():
#     df_voc['voc_day_call_dur{}_sum'.format(i)] = df_voc['phone_no_m'].map(voc_day_call_dur_res[i])

98

##### 每小时通话时长统计 hour

In [21]:
# 每小时通话时长统计
df_voc['call_dur_hour_sum'] = df_voc.groupby(['phone_no_m', 'voc_hour'])['call_dur'].transform('sum')
df_voc['call_dur_hour_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_hour_sum'].transform('max')
df_voc['call_dur_hour_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_hour_sum'].transform('min')
df_voc['call_dur_hour_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_hour_sum'].transform('mean')
df_voc['call_dur_hour_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_hour_sum'].transform('std')


# 每小时与不同的人通话时长统计
df_voc['call_dur_hour_diff_sum'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_hour'])['call_dur'].transform('sum')
df_voc['call_dur_hour_diff_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_hour_diff_sum'].transform('max')
df_voc['call_dur_hour_diff_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_hour_diff_sum'].transform('min')
df_voc['call_dur_hour_diff_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_hour_diff_sum'].transform('mean')
df_voc['call_dur_hour_diff_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_hour_diff_sum'].transform('std')

del df_voc['call_dur_hour_sum'], df_voc['call_dur_hour_diff_sum']
gc.collect()


# 每个phone_no_m在几时的通话时长
voc_hour_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_hour'])['call_dur'].sum().unstack()

for i in df_voc['voc_hour'].unique():
    df_voc['voc_hour{}_call_dur_sum'.format(i)] = df_voc['phone_no_m'].map(voc_hour_call_dur_res[i])

##### 周几通话时长统计 dayofweek

In [22]:
# 周几通话时长统计
df_voc['call_dur_dayofweek_sum'] = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['call_dur'].transform('sum')
df_voc['call_dur_dayofweek_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_sum'].transform('max')
df_voc['call_dur_dayofweek_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_sum'].transform('min')
df_voc['call_dur_dayofweek_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_sum'].transform('mean')
df_voc['call_dur_dayofweek_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_sum'].transform('std')


# 周几与不同的人通话时长统计
df_voc['call_dur_dayofweek_diff_sum'] = df_voc.groupby(['phone_no_m', 'opposite_no_m', 'voc_dayofweek'])['call_dur'].transform('sum')
df_voc['call_dur_dayofweek_diff_sum_max'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_diff_sum'].transform('max')
df_voc['call_dur_dayofweek_diff_sum_min'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_diff_sum'].transform('min')
df_voc['call_dur_dayofweek_diff_sum_mean'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_diff_sum'].transform('mean')
df_voc['call_dur_dayofweek_diff_sum_std'] = df_voc.groupby('phone_no_m')['call_dur_dayofweek_diff_sum'].transform('std')

del df_voc['call_dur_dayofweek_sum'], df_voc['call_dur_dayofweek_diff_sum']
gc.collect()


# 每个phone_no_m在周几的通话时长
voc_dayofweek_call_dur_res = df_voc.groupby(['phone_no_m', 'voc_dayofweek'])['call_dur'].sum().unstack()

for i in df_voc['voc_dayofweek'].unique():
    df_voc['voc_dayofweek{}_call_dur_sum'.format(i)] = df_voc['phone_no_m'].map(voc_dayofweek_call_dur_res[i])

### 短信表

In [23]:
train_sms = pd.read_hdf('../input/train_sms.h5')
test_sms = pd.read_hdf('../input/test_sms.h5')

In [24]:
train_sms['sms_day'] = train_sms['request_datetime'].astype('datetime64').dt.day
train_sms['sms_hour'] = train_sms['request_datetime'].astype('datetime64').dt.hour
train_sms['sms_dayofweek'] = train_sms['request_datetime'].astype('datetime64').dt.dayofweek
train_sms['sms_time'] = (train_sms['sms_day'] - train_sms['sms_day'].min()) * 24 + train_sms['sms_hour']

test_sms['sms_day'] = test_sms['request_datetime'].astype('datetime64').dt.day
test_sms['sms_hour'] = test_sms['request_datetime'].astype('datetime64').dt.hour
test_sms['sms_dayofweek'] = test_sms['request_datetime'].astype('datetime64').dt.dayofweek
test_sms['sms_time'] = (test_sms['sms_day'] - test_sms['sms_day'].min()) * 24 + test_sms['sms_hour']

In [25]:
df_sms = pd.concat([train_sms, test_sms])

In [26]:
df_sms.rename(columns={'calltype_id': 'calltype_id_sms'}, inplace=True)

In [27]:
# 该月总的短信次数
df_sms['sms_count'] = df_sms.groupby('phone_no_m')['phone_no_m'].transform('count')

# 相互发送短信次数
df_sms['sms_count_mutual'] = df_sms.groupby(['phone_no_m', 'opposite_no_m'])['phone_no_m'].transform('count')

In [28]:
# 一天内短信统计量

df_sms['sms_day_count'] = df_sms.groupby(['phone_no_m', 'sms_day'])['phone_no_m'].transform('count')
df_sms['sms_day_count_max'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('max')
df_sms['sms_day_count_min'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('min')
df_sms['sms_day_count_mean'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('mean')
df_sms['sms_day_count_std'] = df_sms.groupby('phone_no_m')['sms_day_count'].transform('std')

del df_sms['sms_day_count']


# 一天内与不同的人短信统计量
df_sms['sms_day_diff_count'] = df_sms.groupby(['phone_no_m', 'opposite_no_m', 'sms_day'])['phone_no_m'].transform('count')
df_sms['sms_day_diff_count_max'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('max')
df_sms['sms_day_diff_count_min'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('min')
df_sms['sms_day_diff_count_mean'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('mean')
df_sms['sms_day_diff_count_std'] = df_sms.groupby('phone_no_m')['sms_day_diff_count'].transform('std')

del df_sms['sms_day_diff_count']
gc.collect()

# 几号短信次数统计
sms_day_res = df_sms.groupby(['phone_no_m', 'sms_day'])['phone_no_m'].count().unstack()

for i in df_sms['sms_day'].unique():
    df_sms['sms_day{}_count'.format(i)] = df_sms['phone_no_m'].map(sms_day_res[i])

In [29]:
# 一小时内短信统计量

df_sms['sms_hour_count'] = df_sms.groupby(['phone_no_m', 'sms_hour'])['phone_no_m'].transform('count')
df_sms['sms_hour_count_max'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('max')
df_sms['sms_hour_count_min'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('min')
df_sms['sms_hour_count_mean'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('mean')
df_sms['sms_hour_count_std'] = df_sms.groupby('phone_no_m')['sms_hour_count'].transform('std')




# 一小时内与不同的人短信统计量
df_sms['sms_hour_diff_count'] = df_sms.groupby(['phone_no_m', 'opposite_no_m', 'sms_hour'])['phone_no_m'].transform('count')
df_sms['sms_hour_diff_count_max'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('max')
df_sms['sms_hour_diff_count_min'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('min')
df_sms['sms_hour_diff_count_mean'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('mean')
df_sms['sms_hour_diff_count_std'] = df_sms.groupby('phone_no_m')['sms_hour_diff_count'].transform('std')

del df_sms['sms_hour_count'], df_sms['sms_hour_diff_count']
gc.collect()

98

In [30]:
# 周几短信统计量

df_sms['sms_dayofweek_count'] = df_sms.groupby(['phone_no_m', 'sms_dayofweek'])['phone_no_m'].transform('count')
df_sms['sms_dayofweek_count_max'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('max')
df_sms['sms_dayofweek_count_min'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('min')
df_sms['sms_dayofweek_count_mean'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('mean')
df_sms['sms_dayofweek_count_std'] = df_sms.groupby('phone_no_m')['sms_dayofweek_count'].transform('std')

# 周几与不同的人短信统计量
df_sms['sms_dayodweek_diff_count'] = df_sms.groupby(['phone_no_m', 'opposite_no_m', 'sms_dayofweek'])['phone_no_m'].transform('count')
df_sms['sms_dayodweek_diff_count_max'] = df_sms.groupby('phone_no_m')['sms_dayodweek_diff_count'].transform('max')
df_sms['sms_dayodweek_diff_count_min'] = df_sms.groupby('phone_no_m')['sms_dayodweek_diff_count'].transform('min')
df_sms['sms_dayodweek_diff_count_mean'] = df_sms.groupby('phone_no_m')['sms_dayodweek_diff_count'].transform('mean')
df_sms['sms_dayodweek_diff_count_std'] = df_sms.groupby('phone_no_m')['sms_dayodweek_diff_count'].transform('std')

del df_sms['sms_dayofweek_count'], df_sms['sms_dayodweek_diff_count']
gc.collect()

# 周几短信次数统计
sms_dayofweek_res = df_sms.groupby(['phone_no_m', 'sms_dayofweek'])['phone_no_m'].count().unstack()

for i in df_sms['sms_dayofweek'].unique():
    df_sms['sms_dayofweek{}_count'.format(i)] = df_sms['phone_no_m'].map(sms_dayofweek_res[i])

### 应用表

In [31]:
train_app = pd.read_hdf('../input/train_app.h5')
test_app = pd.read_hdf('../input/test_app.h5')

In [32]:
df_app = pd.concat([train_app, test_app])

In [33]:
# 用户流量统计

df_app['total_flow'] = df_app.groupby('phone_no_m')['flow'].transform('sum')
df_app['flow_max'] = df_app.groupby('phone_no_m')['flow'].transform('max')
df_app['flow_min'] = df_app.groupby('phone_no_m')['flow'].transform('min')
df_app['flow_mean'] = df_app.groupby('phone_no_m')['flow'].transform('mean')
df_app['flow_std'] = df_app.groupby('phone_no_m')['flow'].transform('std')

# 应用数
df_app['app_count'] = df_app.groupby('phone_no_m')['phone_no_m'].transform('count')

In [34]:
lbl = LabelEncoder()

for f in tqdm(['busi_name']):
    df_app[f] = df_app[f].fillna('NA')
    df_app[f] = lbl.fit_transform(df_app[f].astype(str))

100%|████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.89it/s]


In [35]:
# 用户在每个app上的流量统计

df_app['busi_name_total_flow'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('sum')
df_app['busi_name_flow_max'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('max')
df_app['busi_name_flow_min'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('min')
df_app['busi_name_flow_mean'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('mean')
df_app['busi_name_flow_std'] = df_app.groupby(['phone_no_m', 'busi_name'])['flow'].transform('std')

### 合并数据

In [36]:
df_user.shape, df_voc.shape, df_sms.shape, df_app.shape

((8151, 16), (985666, 181), (1319506, 72), (512100, 15))

In [37]:
print(df_user.columns.tolist())

['arpu_202004', 'city_name', 'county_name', 'idcard_cnt', 'label', 'phone_no_m', 'idcard_cnt*arpu_202004', 'city_name_count', 'county_name_count', 'idcard_cnt_count', 'city_name_idcard_cnt_max', 'city_name_idcard_cnt_min', 'city_name_idcard_cnt_median', 'county_name_idcard_cnt_max', 'county_name_idcard_cnt_min', 'county_name_idcard_cnt_median']


In [38]:
print(df_voc.columns.tolist())

['phone_no_m', 'opposite_no_m', 'calltype_id', 'start_datetime', 'call_dur', 'city_name', 'county_name', 'imei_m', 'voc_day', 'voc_hour', 'voc_dayofweek', 'voc_time', 'voc_phone_nunique', 'voc_count', 'call_dur_sum', 'voc_count_mutual', 'call_dur_sum_mutual', 'voc_day_count_max', 'voc_day_count_min', 'voc_day_count_mean', 'voc_day_count_std', 'voc_day_diff_count_max', 'voc_day_diff_count_min', 'voc_day_diff_count_mean', 'voc_day_diff_count_std', 'voc_day22_count', 'voc_day23_count', 'voc_day1_count', 'voc_day3_count', 'voc_day9_count', 'voc_day6_count', 'voc_day10_count', 'voc_day26_count', 'voc_day27_count', 'voc_day13_count', 'voc_day28_count', 'voc_day30_count', 'voc_day20_count', 'voc_day16_count', 'voc_day19_count', 'voc_day21_count', 'voc_day29_count', 'voc_day17_count', 'voc_day7_count', 'voc_day14_count', 'voc_day15_count', 'voc_day5_count', 'voc_day2_count', 'voc_day12_count', 'voc_day8_count', 'voc_day18_count', 'voc_day31_count', 'voc_day4_count', 'voc_day24_count', 'voc_day

In [39]:
voc_cols = [i for i in df_voc.columns if i not in ['opposite_no_m', 'start_datetime', 'city_name', 'county_name', 'imei_m']]
df_voc = df_voc[voc_cols].drop_duplicates(subset=['phone_no_m'])

In [40]:
print(df_sms.columns.tolist())

['phone_no_m', 'opposite_no_m', 'calltype_id_sms', 'request_datetime', 'sms_day', 'sms_hour', 'sms_dayofweek', 'sms_time', 'sms_count', 'sms_count_mutual', 'sms_day_count_max', 'sms_day_count_min', 'sms_day_count_mean', 'sms_day_count_std', 'sms_day_diff_count_max', 'sms_day_diff_count_min', 'sms_day_diff_count_mean', 'sms_day_diff_count_std', 'sms_day1_count', 'sms_day2_count', 'sms_day3_count', 'sms_day4_count', 'sms_day5_count', 'sms_day6_count', 'sms_day7_count', 'sms_day8_count', 'sms_day9_count', 'sms_day10_count', 'sms_day11_count', 'sms_day12_count', 'sms_day13_count', 'sms_day14_count', 'sms_day15_count', 'sms_day16_count', 'sms_day17_count', 'sms_day18_count', 'sms_day19_count', 'sms_day20_count', 'sms_day21_count', 'sms_day22_count', 'sms_day23_count', 'sms_day24_count', 'sms_day25_count', 'sms_day26_count', 'sms_day27_count', 'sms_day28_count', 'sms_day29_count', 'sms_day30_count', 'sms_day31_count', 'sms_hour_count_max', 'sms_hour_count_min', 'sms_hour_count_mean', 'sms_ho

In [41]:
sms_cols = [i for i in df_sms.columns if i not in ['opposite_no_m', 'request_datetime']]
df_sms = df_sms[sms_cols].drop_duplicates(subset=['phone_no_m'])

In [42]:
print(df_app.columns.tolist())

['phone_no_m', 'busi_name', 'flow', 'month_id', 'total_flow', 'flow_max', 'flow_min', 'flow_mean', 'flow_std', 'app_count', 'busi_name_total_flow', 'busi_name_flow_max', 'busi_name_flow_min', 'busi_name_flow_mean', 'busi_name_flow_std']


In [43]:
app_cols = [i for i in df_app.columns if i not in ['month_id']]
df_app = df_app[app_cols].drop_duplicates(subset=['phone_no_m'])

In [44]:
df = pd.merge(df_user, df_voc, how='left', on='phone_no_m')
df = pd.merge(df, df_sms, how='left', on='phone_no_m')
df = pd.merge(df, df_app, how='left', on='phone_no_m')

In [45]:
del df_user, df_voc, df_sms, df_app
gc.collect()

118

In [46]:
print(df.columns.tolist())

['arpu_202004', 'city_name', 'county_name', 'idcard_cnt', 'label', 'phone_no_m', 'idcard_cnt*arpu_202004', 'city_name_count', 'county_name_count', 'idcard_cnt_count', 'city_name_idcard_cnt_max', 'city_name_idcard_cnt_min', 'city_name_idcard_cnt_median', 'county_name_idcard_cnt_max', 'county_name_idcard_cnt_min', 'county_name_idcard_cnt_median', 'calltype_id', 'call_dur', 'voc_day', 'voc_hour', 'voc_dayofweek', 'voc_time', 'voc_phone_nunique', 'voc_count', 'call_dur_sum', 'voc_count_mutual', 'call_dur_sum_mutual', 'voc_day_count_max', 'voc_day_count_min', 'voc_day_count_mean', 'voc_day_count_std', 'voc_day_diff_count_max', 'voc_day_diff_count_min', 'voc_day_diff_count_mean', 'voc_day_diff_count_std', 'voc_day22_count', 'voc_day23_count', 'voc_day1_count', 'voc_day3_count', 'voc_day9_count', 'voc_day6_count', 'voc_day10_count', 'voc_day26_count', 'voc_day27_count', 'voc_day13_count', 'voc_day28_count', 'voc_day30_count', 'voc_day20_count', 'voc_day16_count', 'voc_day19_count', 'voc_day21

In [47]:
df_train = df[df.label.notna()]
df_test = df[df.label.isna()]

df_train.shape, df_test.shape

((6106, 273), (2045, 273))

In [48]:
df_train_train = df_train[df_train['voc_day'] <= 23]
df_train_valid = df_train[df_train['voc_day'] > 23]

X_train = df_train_train.drop('label', axis=1)
y_train = df_train_train['label']
X_valid = df_train_valid.drop('label', axis=1)
y_valid = df_train_valid['label']

In [49]:
train_cols = [i for i in X_train if i not in ['phone_no_m', 'label']]

In [50]:
params = {'objective': 'binary',
          'boosting': 'gbdt',
          'metric': 'auc',
          'learning_rate': 0.1,
          'num_leaves': 31,
          'lambda_l1': 0,
          'lambda_l2': 1,
          'min_data_in_leaf': 20,
          'is_unbalance': True,
          'max_depth': -1,
          'seed': 2020}

In [51]:
use_cols = []
useless_cols = []

for i in train_cols:
    print(i)
    
    lgb_train = lgb.Dataset(X_train[[i]].values, y_train) 
    lgb_eval= lgb.Dataset(X_valid[[i]].values, y_valid, reference=lgb_train)
    lgb_test = lgb.train(params,
                         lgb_train,
                         num_boost_round=1000,
                         valid_sets=[lgb_eval, lgb_train],
                         early_stopping_rounds=50,
                         verbose_eval=1)
    
    print('*' * 5)
    print(lgb_test.best_score['valid_0']['auc'])
    if lgb_test.best_score['valid_0']['auc'] > 0.52:
        use_cols.append(i)
    else:
        useless_cols.append(i)
    print('*' * 20)
    print('\n')

arpu_202004
[1]	training's auc: 0.702378	valid_0's auc: 0.63099
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.703723	valid_0's auc: 0.628047
[3]	training's auc: 0.706151	valid_0's auc: 0.625016
[4]	training's auc: 0.70824	valid_0's auc: 0.623991
[5]	training's auc: 0.712133	valid_0's auc: 0.630661
[6]	training's auc: 0.712468	valid_0's auc: 0.629535
[7]	training's auc: 0.716159	valid_0's auc: 0.627363
[8]	training's auc: 0.716754	valid_0's auc: 0.627256
[9]	training's auc: 0.724177	valid_0's auc: 0.624144
[10]	training's auc: 0.7253	valid_0's auc: 0.623848
[11]	training's auc: 0.72616	valid_0's auc: 0.622018
[12]	training's auc: 0.726034	valid_0's auc: 0.620124
[13]	training's auc: 0.727831	valid_0's auc: 0.622874
[14]	training's auc: 0.729042	valid_0's auc: 0.623356
[15]	training's auc: 0.729523	valid_0's auc: 0.622044
[16]	training's auc: 0.730267	valid_0's auc: 0.621787
[17]	training's auc: 0.730734	valid_0's auc: 0.620866
[18]	training's auc: 0.

[42]	training's auc: 0.753598	valid_0's auc: 0.611029
[43]	training's auc: 0.753586	valid_0's auc: 0.6114
[44]	training's auc: 0.753574	valid_0's auc: 0.610742
[45]	training's auc: 0.753753	valid_0's auc: 0.611517
[46]	training's auc: 0.753986	valid_0's auc: 0.611712
[47]	training's auc: 0.754134	valid_0's auc: 0.612793
[48]	training's auc: 0.754276	valid_0's auc: 0.612637
[49]	training's auc: 0.754477	valid_0's auc: 0.611895
[50]	training's auc: 0.754565	valid_0's auc: 0.612318
[51]	training's auc: 0.7547	valid_0's auc: 0.612344
[52]	training's auc: 0.754786	valid_0's auc: 0.61334
Early stopping, best iteration is:
[2]	training's auc: 0.731584	valid_0's auc: 0.614792
*****
0.6147916666666666
********************


idcard_cnt
[1]	training's auc: 0.688318	valid_0's auc: 0.650553
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.688318	valid_0's auc: 0.650553
[3]	training's auc: 0.689453	valid_0's auc: 0.653587
[4]	training's auc: 0.689453	valid_0's auc: 

[26]	training's auc: 0.644039	valid_0's auc: 0.601159
[27]	training's auc: 0.644039	valid_0's auc: 0.601159
[28]	training's auc: 0.644039	valid_0's auc: 0.601159
[29]	training's auc: 0.644039	valid_0's auc: 0.601159
[30]	training's auc: 0.644039	valid_0's auc: 0.601159
[31]	training's auc: 0.644039	valid_0's auc: 0.601159
[32]	training's auc: 0.644039	valid_0's auc: 0.601159
[33]	training's auc: 0.644039	valid_0's auc: 0.601159
[34]	training's auc: 0.644039	valid_0's auc: 0.601159
[35]	training's auc: 0.644039	valid_0's auc: 0.601159
[36]	training's auc: 0.644039	valid_0's auc: 0.601159
[37]	training's auc: 0.644041	valid_0's auc: 0.602956
[38]	training's auc: 0.644041	valid_0's auc: 0.602956
[39]	training's auc: 0.644041	valid_0's auc: 0.602956
[40]	training's auc: 0.644041	valid_0's auc: 0.602956
[41]	training's auc: 0.644041	valid_0's auc: 0.602956
[42]	training's auc: 0.644041	valid_0's auc: 0.602956
[43]	training's auc: 0.644041	valid_0's auc: 0.602956
[44]	training's auc: 0.64404

[11]	training's auc: 0.624972	valid_0's auc: 0.57112
[12]	training's auc: 0.624972	valid_0's auc: 0.57112
[13]	training's auc: 0.624972	valid_0's auc: 0.57112
[14]	training's auc: 0.624972	valid_0's auc: 0.57112
[15]	training's auc: 0.624972	valid_0's auc: 0.57112
[16]	training's auc: 0.624972	valid_0's auc: 0.57112
[17]	training's auc: 0.624972	valid_0's auc: 0.57112
[18]	training's auc: 0.624972	valid_0's auc: 0.57112
[19]	training's auc: 0.624972	valid_0's auc: 0.57112
[20]	training's auc: 0.624972	valid_0's auc: 0.57112
[21]	training's auc: 0.624972	valid_0's auc: 0.57112
[22]	training's auc: 0.624972	valid_0's auc: 0.57112
[23]	training's auc: 0.624972	valid_0's auc: 0.57112
[24]	training's auc: 0.624972	valid_0's auc: 0.57112
[25]	training's auc: 0.624972	valid_0's auc: 0.57112
[26]	training's auc: 0.624972	valid_0's auc: 0.57112
[27]	training's auc: 0.624972	valid_0's auc: 0.57112
[28]	training's auc: 0.624972	valid_0's auc: 0.57112
[29]	training's auc: 0.624972	valid_0's auc: 0

county_name_idcard_cnt_max
[1]	training's auc: 0.595137	valid_0's auc: 0.559775
Training until validation scores don't improve for 50 rounds
[2]	training's auc: 0.595137	valid_0's auc: 0.559775
[3]	training's auc: 0.595137	valid_0's auc: 0.559775
[4]	training's auc: 0.595137	valid_0's auc: 0.559775
[5]	training's auc: 0.595169	valid_0's auc: 0.559469
[6]	training's auc: 0.595169	valid_0's auc: 0.559469
[7]	training's auc: 0.595169	valid_0's auc: 0.559469
[8]	training's auc: 0.595169	valid_0's auc: 0.559469
[9]	training's auc: 0.595169	valid_0's auc: 0.559469
[10]	training's auc: 0.595169	valid_0's auc: 0.559469
[11]	training's auc: 0.595169	valid_0's auc: 0.559469
[12]	training's auc: 0.595169	valid_0's auc: 0.559469
[13]	training's auc: 0.595169	valid_0's auc: 0.559469
[14]	training's auc: 0.595169	valid_0's auc: 0.559469
[15]	training's auc: 0.595169	valid_0's auc: 0.559469
[16]	training's auc: 0.595169	valid_0's auc: 0.559469
[17]	training's auc: 0.595169	valid_0's auc: 0.559469
[18

...

[9]	training's auc: 0.720958	valid_0's auc: 0.616543
[10]	training's auc: 0.722833	valid_0's auc: 0.618597
[11]	training's auc: 0.722142	valid_0's auc: 0.618464
[12]	training's auc: 0.724089	valid_0's auc: 0.617188
[13]	training's auc: 0.726296	valid_0's auc: 0.618073
[14]	training's auc: 0.727009	valid_0's auc: 0.618685
[15]	training's auc: 0.727414	valid_0's auc: 0.61986
[16]	training's auc: 0.727939	valid_0's auc: 0.618457
[17]	training's auc: 0.728714	valid_0's auc: 0.617699
[18]	training's auc: 0.729297	valid_0's auc: 0.617816
[19]	training's auc: 0.730137	valid_0's auc: 0.617676
[20]	training's auc: 0.730272	valid_0's auc: 0.618558
[21]	training's auc: 0.730631	valid_0's auc: 0.618685
[22]	training's auc: 0.731143	valid_0's auc: 0.619567
[23]	training's auc: 0.731429	valid_0's auc: 0.619339
[24]	training's auc: 0.73181	valid_0's auc: 0.619326
[25]	training's auc: 0.731939	valid_0's auc: 0.619909
[26]	training's auc: 0.732628	valid_0's auc: 0.618353
[27]	training's auc: 0.732879	v

[23]	training's auc: 0.702931	valid_0's auc: 0.582852
[24]	training's auc: 0.703691	valid_0's auc: 0.582187
[25]	training's auc: 0.704004	valid_0's auc: 0.583548
[26]	training's auc: 0.704374	valid_0's auc: 0.58293
[27]	training's auc: 0.704743	valid_0's auc: 0.582995
[28]	training's auc: 0.704888	valid_0's auc: 0.584111
[29]	training's auc: 0.705057	valid_0's auc: 0.584769
[30]	training's auc: 0.706366	valid_0's auc: 0.583366
[31]	training's auc: 0.706311	valid_0's auc: 0.583405
[32]	training's auc: 0.706912	valid_0's auc: 0.582855
[33]	training's auc: 0.707122	valid_0's auc: 0.583809
[34]	training's auc: 0.707469	valid_0's auc: 0.583424
[35]	training's auc: 0.707574	valid_0's auc: 0.583008
[36]	training's auc: 0.708039	valid_0's auc: 0.582422
[37]	training's auc: 0.708183	valid_0's auc: 0.582474
[38]	training's auc: 0.708962	valid_0's auc: 0.582314
[39]	training's auc: 0.709045	valid_0's auc: 0.58292
[40]	training's auc: 0.709395	valid_0's auc: 0.583034
[41]	training's auc: 0.709293	

[9]	training's auc: 0.677856	valid_0's auc: 0.581338
[10]	training's auc: 0.679521	valid_0's auc: 0.578939
[11]	training's auc: 0.681878	valid_0's auc: 0.582259
[12]	training's auc: 0.684089	valid_0's auc: 0.581885
[13]	training's auc: 0.684171	valid_0's auc: 0.580713
[14]	training's auc: 0.684745	valid_0's auc: 0.580417
[15]	training's auc: 0.685336	valid_0's auc: 0.580166
[16]	training's auc: 0.686747	valid_0's auc: 0.578734
[17]	training's auc: 0.687082	valid_0's auc: 0.57709
[18]	training's auc: 0.687856	valid_0's auc: 0.57737
[19]	training's auc: 0.689032	valid_0's auc: 0.576419
[20]	training's auc: 0.689921	valid_0's auc: 0.575469
[21]	training's auc: 0.690353	valid_0's auc: 0.576523
[22]	training's auc: 0.691147	valid_0's auc: 0.574704
[23]	training's auc: 0.691414	valid_0's auc: 0.574762
[24]	training's auc: 0.691303	valid_0's auc: 0.576097
[25]	training's auc: 0.693035	valid_0's auc: 0.575518
[26]	training's auc: 0.693633	valid_0's auc: 0.575352
[27]	training's auc: 0.694052	v

[35]	training's auc: 0.717007	valid_0's auc: 0.600977
[36]	training's auc: 0.71725	valid_0's auc: 0.601185
[37]	training's auc: 0.717787	valid_0's auc: 0.599938
[38]	training's auc: 0.718086	valid_0's auc: 0.600192
[39]	training's auc: 0.718457	valid_0's auc: 0.599349
[40]	training's auc: 0.718788	valid_0's auc: 0.599463
[41]	training's auc: 0.719402	valid_0's auc: 0.600527
[42]	training's auc: 0.719646	valid_0's auc: 0.601504
[43]	training's auc: 0.719893	valid_0's auc: 0.601673
[44]	training's auc: 0.72034	valid_0's auc: 0.599665
[45]	training's auc: 0.720435	valid_0's auc: 0.599359
[46]	training's auc: 0.721271	valid_0's auc: 0.600928
[47]	training's auc: 0.721461	valid_0's auc: 0.600872
[48]	training's auc: 0.72164	valid_0's auc: 0.601895
[49]	training's auc: 0.721912	valid_0's auc: 0.601042
[50]	training's auc: 0.722114	valid_0's auc: 0.601035
[51]	training's auc: 0.722283	valid_0's auc: 0.601589
[52]	training's auc: 0.722431	valid_0's auc: 0.600736
[53]	training's auc: 0.722607	v

[19]	training's auc: 0.687593	valid_0's auc: 0.553024
[20]	training's auc: 0.688504	valid_0's auc: 0.553245
[21]	training's auc: 0.689222	valid_0's auc: 0.554154
[22]	training's auc: 0.690143	valid_0's auc: 0.553906
[23]	training's auc: 0.690286	valid_0's auc: 0.555703
[24]	training's auc: 0.690821	valid_0's auc: 0.556722
[25]	training's auc: 0.690767	valid_0's auc: 0.556006
[26]	training's auc: 0.691373	valid_0's auc: 0.5557
[27]	training's auc: 0.691676	valid_0's auc: 0.554287
[28]	training's auc: 0.691928	valid_0's auc: 0.553376
[29]	training's auc: 0.692408	valid_0's auc: 0.554085
[30]	training's auc: 0.692647	valid_0's auc: 0.553734
[31]	training's auc: 0.692853	valid_0's auc: 0.553597
[32]	training's auc: 0.693137	valid_0's auc: 0.553737
[33]	training's auc: 0.693524	valid_0's auc: 0.553171
[34]	training's auc: 0.693888	valid_0's auc: 0.552565
[35]	training's auc: 0.694089	valid_0's auc: 0.552669
[36]	training's auc: 0.694382	valid_0's auc: 0.553132
[37]	training's auc: 0.694798	

In [52]:
print(use_cols)

['arpu_202004', 'city_name', 'county_name', 'idcard_cnt', 'idcard_cnt*arpu_202004', 'city_name_count', 'county_name_count', 'idcard_cnt_count', 'city_name_idcard_cnt_max', 'city_name_idcard_cnt_min', 'city_name_idcard_cnt_median', 'county_name_idcard_cnt_max', 'county_name_idcard_cnt_min', 'county_name_idcard_cnt_median', 'calltype_id', 'voc_dayofweek', 'voc_phone_nunique', 'voc_count', 'call_dur_sum', 'voc_count_mutual', 'call_dur_sum_mutual', 'voc_day_count_max', 'voc_day_count_min', 'voc_day_count_mean', 'voc_day_count_std', 'voc_day_diff_count_max', 'voc_day_diff_count_mean', 'voc_day_diff_count_std', 'voc_day22_count', 'voc_day23_count', 'voc_day1_count', 'voc_day3_count', 'voc_day9_count', 'voc_day6_count', 'voc_day10_count', 'voc_day26_count', 'voc_day27_count', 'voc_day13_count', 'voc_day28_count', 'voc_day30_count', 'voc_day20_count', 'voc_day16_count', 'voc_day19_count', 'voc_day21_count', 'voc_day29_count', 'voc_day17_count', 'voc_day7_count', 'voc_day14_count', 'voc_day15_c

In [53]:
print(useless_cols)

['call_dur', 'voc_day', 'voc_hour', 'voc_time', 'voc_day_diff_count_min', 'voc_time_diff_count_min', 'voc_hour_diff_count_min', 'voc_dayofweek_diff_count_min', 'voc_hour1_call_dur_sum', 'voc_hour3_call_dur_sum', 'voc_hour4_call_dur_sum', 'sms_day', 'sms_dayofweek', 'sms_time', 'sms_day_diff_count_min', 'sms_hour_count_min', 'sms_hour_diff_count_min', 'sms_dayodweek_diff_count_min']


In [54]:
params = {'objective': 'binary',
          'boosting': 'gbdt',
          'metric': 'binary_logloss',
          'learning_rate': 0.1,
          'num_leaves': 31,
          'lambda_l1': 0,
          'lambda_l2': 1,
          'min_data_in_leaf': 20,
          'is_unbalance': True,
          'max_depth': -1,
          'seed': 2020}

In [55]:
lgb_train = lgb.Dataset(X_train[use_cols].values, y_train) 

lgb_eval= lgb.Dataset(X_valid[use_cols].values, y_valid, reference=lgb_train)  

print('Start training...')

# train

lgb_val_0 = lgb.train(params,
                      lgb_train,
                      num_boost_round=10000,
                      valid_sets=[lgb_eval, lgb_train],
                      early_stopping_rounds=100,
                      verbose_eval=10)

Start training...
Training until validation scores don't improve for 100 rounds
[10]	training's binary_logloss: 0.242708	valid_0's binary_logloss: 0.294924
[20]	training's binary_logloss: 0.16787	valid_0's binary_logloss: 0.270887
[30]	training's binary_logloss: 0.12152	valid_0's binary_logloss: 0.253891
[40]	training's binary_logloss: 0.0880459	valid_0's binary_logloss: 0.239276
[50]	training's binary_logloss: 0.0638404	valid_0's binary_logloss: 0.229185
[60]	training's binary_logloss: 0.0470831	valid_0's binary_logloss: 0.225236
[70]	training's binary_logloss: 0.0349764	valid_0's binary_logloss: 0.223213
[80]	training's binary_logloss: 0.0263872	valid_0's binary_logloss: 0.222097
[90]	training's binary_logloss: 0.020201	valid_0's binary_logloss: 0.222671
[100]	training's binary_logloss: 0.0156622	valid_0's binary_logloss: 0.224406
[110]	training's binary_logloss: 0.0123387	valid_0's binary_logloss: 0.227872
[120]	training's binary_logloss: 0.00989242	valid_0's binary_logloss: 0.23173

In [56]:
# 验证集结果
X_valid['prob'] = lgb_val_0.predict(X_valid[use_cols])
X_valid['pred'] = np.where(X_valid['prob'] > 0.4735, 1, 0)

f1_04735 = np.round(f1_score(y_valid, X_valid['pred']), 4)
auc_04735 = roc_auc_score(y_valid, X_valid['prob'])

print('f1_04735: ', f1_04735)
print('auc_04735: ', auc_04735)

f1_04735:  0.5523
auc_04735:  0.8559244791666667


In [57]:
# 验证集结果
X_valid['prob'] = lgb_val_0.predict(X_valid[use_cols])
X_valid['pred'] = np.where(X_valid['prob'] > 0.5, 1, 0)

f1_05 = np.round(f1_score(y_valid, X_valid['pred']), 4)
auc_05 = roc_auc_score(y_valid, X_valid['prob'])

print('f1_05: ', f1_05)
print('auc_05: ', auc_05)

f1_05:  0.5617
auc_05:  0.8559244791666667


In [58]:
lgb_train_all = lgb.Dataset(df_train[use_cols].values, df_train['label'])   

print('Start training...')

# train

lgb_model = lgb.train(params,
                      lgb_train_all,
                      num_boost_round=lgb_val_0.best_iteration + 20)

Start training...


In [59]:
df_test['label'] = np.where(lgb_model.predict(df_test[use_cols]) > 0.5, 1, 0)
df_test[['phone_no_m', 'label']].to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), f1_05), index=False)

In [60]:
df_test['label'] = np.where(lgb_model.predict(df_test[use_cols]) > 0.4735, 1, 0)
df_test[['phone_no_m', 'label']].to_csv('../sub/sub_{}_{}.csv'.format(time.strftime('%Y%m%d'), f1_04735), index=False)